<a href="https://colab.research.google.com/github/LilianYou/Sea_Hero_Quest/blob/main/Ordinal_Regression_Analysis_Shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Characterizing self-reported navigation ability using demographic information

##Ordinal Logistic Regression

### read data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/My Drive/Data/

/content/drive/.shortcut-targets-by-id/1pS1W_QFv_TreubUVne2-kLNgsBSGwhhw/Data


In [4]:
df = pd.read_csv(r'2019user_cleaned_3121970.csv')[['age','education','gender','hand','home_environment','navigating_skills','sleep','travel_time']]

In [5]:
df.head()

,age,education,gender,hand,home_environment,navigating_skills,sleep,travel_time
0,68.0,college,f,right,city,bad,8.0,less-30-mins
1,21.0,university,f,right,suburbs,good,7.0,30-mins-to-1-hour
2,36.0,university,m,right,rural,very-good,5.0,30-mins-to-1-hour
3,19.0,high-school,m,right,mixed,good,8.0,less-30-mins
4,61.0,university,m,right,suburbs,good,7.0,less-30-mins


In [6]:
df_model = df.copy()
recode_map = {"gender": {'f': 0,'m': 1},
                   "hand": {'right': 0,'left': 1},
                   "education":{"high-school":0, "no-formal":0,"college":1,"university":1},
                   "travel_time":{"less-30-mins":0, "30-mins-to-1-hour":1, "hour-plus":2},
                   "home_environment":{"rural":0,"suburbs":1,"mixed":1,"city":2}}
df_model = df_model.replace(recode_map)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df_model[['age','sleep','home_environment','travel_time']] = sc.fit_transform(df_model[['age','sleep','home_environment','travel_time']])

In [8]:
df_model['navigating_skills'] = pd.Categorical(df_model['navigating_skills'], 
                                 categories=['very-bad','bad','good','very-good'], 
                                 ordered=True)

In [9]:
df_model.head()

,age,education,gender,hand,home_environment,navigating_skills,sleep,travel_time
0,2.020202,1,0,0,1.275715,bad,0.845968,-1.074199
1,-1.210484,1,0,0,-0.198615,good,0.011091,0.184414
2,-0.179414,1,1,0,-1.672944,very-good,-1.658662,0.184414
3,-1.347960,0,1,0,-0.198615,good,0.845968,-1.074199
4,1.539036,1,1,0,-0.198615,good,0.011091,-1.074199


# Ordinal Regression Model


**[Ordinal Regression](https://www.statsmodels.org/devel/generated/statsmodels.miscmodels.ordinal_model.OrderedModel.html)**

[Sample Code](https://www.statsmodels.org/devel/examples/notebooks/generated/ordinal_regression.html)

[Logistic Regression](https://stats.idre.ucla.edu/r/dae/ordinal-logistic-regression/)

[Probit Regression](https://stats.idre.ucla.edu/stata/dae/probit-regression/)

[Effect size for logistic regression](https://www.theanalysisfactor.com/effect-size-statistics-logistic-regression/)


In [10]:
pip install statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
pip install --upgrade --no-deps statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 10.2 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.12.2
    Uninstalling statsmodels-0.12.2:
      Successfully uninstalled statsmodels-0.12.2


In [12]:
import scipy.stats as stats

from statsmodels.miscmodels.ordinal_model import OrderedModel
from statsmodels.stats.anova import anova_lm

In [13]:
%%time
age_gender_logit_al = OrderedModel.from_formula("navigating_skills ~ age+gender+age:gender+education+gender:education+travel_time+gender:travel_time+education:travel_time+hand+sleep+home_environment", df_model,
                                      distr='logit')
res_a_g_logit_al = age_gender_logit_al.fit(method='bfgs')
print(res_a_g_logit_al.summary())

Optimization terminated successfully.
         Current function value: 0.988688
         Iterations: 53
         Function evaluations: 54
         Gradient evaluations: 54
                             OrderedModel Results                             
Dep. Variable:      navigating_skills   Log-Likelihood:            -7.6290e+05
Model:                   OrderedModel   AIC:                         1.526e+06
Method:            Maximum Likelihood   BIC:                         1.526e+06
Date:                Sat, 24 Sep 2022                                         
Time:                        16:59:30                                         
No. Observations:              771628                                         
Df Residuals:                  771614                                         
Df Model:                          14                                         
                            coef    std err          z      P>|z|      [0.025      0.975]
---------------------------

In [14]:
sum = pd.DataFrame.from_records(res_a_g_logit_al.summary().tables[1].data)
header = sum.iloc[0] # grab the first row for the header
sum = sum[1:] # take the data less the header row
sum.columns = header
sum

,,coef,std err,z,P>|z|,[0.025,0.975]
1,age,0.0589,0.003,18.643,0.000,0.053,0.065
2,gender,0.8512,0.009,100.122,0.000,0.835,0.868
3,age:gender,0.0705,0.005,15.624,0.000,0.062,0.079
4,education,0.0445,0.007,6.112,0.000,0.030,0.059
5,gender:education,0.1381,0.010,13.873,0.000,0.119,0.158
6,travel_time,0.3244,0.005,66.721,0.000,0.315,0.334
7,gender:travel_time,-0.0819,0.005,-17.977,0.000,-0.091,-0.073
8,education:travel_time,-0.1377,0.005,-27.982,0.000,-0.147,-0.128
9,hand,-0.0244,0.007,-3.274,0.001,-0.039,-0.010
10,sleep,0.0546,0.002,24.108,0.000,0.050,0.059


In [15]:
sum['coef'] = sum['coef'].astype('float')

## Calculate & Interpret Effect Size

In [16]:
import numpy as np
np.exp(sum['coef'])

1     1.060669
2     2.342456
3     1.073045
4     1.045505
5     1.148090
6     1.383200
7     0.921364
8     0.871360
9     0.975895
10    1.056118
11    0.961558
12    0.031117
13    2.041531
14    2.762677
Name: coef, dtype: float64

In [17]:
import numpy as np
print('age:', np.exp(0.787))
print('gender:', np.exp(0.787))
print('gender-l:', np.exp(0.773))
print('gender-u:', np.exp(0.801))
print('age by gender:', np.exp(0.137))
print('age by gender-l:', np.exp(0.129))
print('age by gender-u:', np.exp(0.145))

age: 2.196796142353235
gender: 2.196796142353235
gender-l: 2.1662552812206535
gender-u: 2.2277675825624406
age by gender: 1.1468281485203982
age by gender-l: 1.1376901241657316
age by gender-u: 1.1560395702680215


[Model Preformance Parmeters](https://www.statsmodels.org/dev/generated/statsmodels.discrete.discrete_model.LogitResults.html)

[Pseduo R square](http://thestatsgeek.com/2014/02/08/r-squared-in-logistic-regression/)


In [18]:
llm1 = res_a_g_logit_al.llf

In [19]:
llm0 = res_a_g_logit_al.llnull

In [20]:
res_a_g_logit_al.llr

55302.18050145591

In [21]:
res_a_g_logit_al.llr_pvalue

0.0

In [22]:
from scipy.stats.distributions import chi2
def likelihood_ratio(llmin, llmax):
    return(2*(llmax-llmin))


LR = likelihood_ratio(llm0,llm1)

print("Chi-square:", LR)
print("p:",chi2.sf(LR, 12)) # L2 has 12 DoF more than L1

Chi-square: 55302.18050145591
p: 0.0


In [23]:
res_a_g_logit_al.prsquared

0.034977010674459574